In [2]:
import placekey as pk
from placekey.api import PlacekeyAPI

In [3]:
placekey_api_key = "Q8J3zrMOuMwyQokDr8x3YX47I7CfXMOQ"
pk_api = PlacekeyAPI(placekey_api_key)
pk_api.lookup_placekey(latitude=37.7371, longitude=-122.44283)

{'query_id': '0', 'placekey': '@5vg-82n-kzz'}

In [4]:
place = {
   "street_address": "598 Portola Dr",
   "city": "San Francisco",
   "region": "CA",
   "postal_code": "94131",
   "iso_country_code": "US"
}
pk_api.lookup_placekey(**place, strict_address_match=True)

{'query_id': '0', 'placekey': '227@5vg-82n-pgk'}